In [1]:
import logging
logging.getLogger("py4j").setLevel(logging.ERROR)

-sandbox

# Spark+AI Summit 2020
## Koalas: pandas on Apache Spark
*Notebook available here:* [bit.ly/koalas_summit_2020]()
<div style="text-align: center; line-height: 10; padding-top: 20px;">
  <img src="https://raw.githubusercontent.com/databricks/koalas/master/Koalas-logo.png" width="220"/>
</div>

The following tutorial takes us on a deep dive of the Koalas API. We will be looking at a dataset containing the occurrence of flora and fauna species in Austraila on a 5 x 5 km scale between the years of 2010-2020. 

The dataset used in this tutorial was obtained from the [Global Biodiversity Information Facility](https://www.gbif.org/). Prior to download the dataset was filtered to occurences of species in Australia from the start of 2010 to May 2020.

**Outline**
1. Load Data
2. Exploratory Data Analysis
3. Analysis of Koala Numbers in Australia
4. Forecasting with Prophet

**Libraries:**
* `koalas==1.0.0`
* `fbprophet==0.6`

**Tested on:**
* `DBR 7.0 ML Beta`

**Data Citation:**
* *Nature data from Australia [2010-2020]. Occurrence dataset https://doi.org/10.15468/dl.van7jk accessed via [GBIF.org](https://www.gbif.org/) on 2020-05-13.*

# 1. Loading Data

In [4]:
# Use notebook scoped libs
%pip install koalas==1.0.0 fbprophet==0.6

Python interpreter will be restarted.
Requirement already satisfied: koalas==1.0.0 in /local_disk0/pythonVirtualEnvDirs/virtualEnv-4f166160-0e58-461f-9476-d73ade108c55/lib/python3.7/site-packages (1.0.0)
Requirement already satisfied: fbprophet==0.6 in /local_disk0/pythonVirtualEnvDirs/virtualEnv-4f166160-0e58-461f-9476-d73ade108c55/lib/python3.7/site-packages (0.6)
Requirement already satisfied: pandas>=0.23.2 in /local_disk0/pythonVirtualEnvDirs/virtualEnv-4f166160-0e58-461f-9476-d73ade108c55/lib/python3.7/site-packages (from koalas==1.0.0) (1.0.1)
Requirement already satisfied: numpy>=1.14 in /local_disk0/pythonVirtualEnvDirs/virtualEnv-4f166160-0e58-461f-9476-d73ade108c55/lib/python3.7/site-packages (from koalas==1.0.0) (1.18.1)
Requirement already satisfied: matplotlib>=3.0.0 in /local_disk0/pythonVirtualEnvDirs/virtualEnv-4f166160-0e58-461f-9476-d73ade108c55/lib/python3.7/site-packages (from koalas==1.0.0) (3.1.3)
Requirement already satisfied: pyarrow>=0.10 in /local_disk0/pythonVirtualEnvDirs/virtualEnv-4f166160-0e58-461f-9476-d73ade108c55/lib/python3.7/site-packages (from koalas==1.0.0) (0.15.1)
Requirement already satisfied: Cython>=0.22 in /local_disk0/pythonVirtualEnvDirs/virtualEnv-4f166160-0e58-461f-9476-d73ade108c55/lib/python3.7/site-packages (from fbprophet==0.6) (0.29.15)
Requirement already satisfied: cmdstanpy==0.4 in /local_disk0/pythonVirtualEnvDirs/virtualEnv-4f166160-0e58-461f-9476-d73ade108c55/lib/python3.7/site-packages (from fbprophet==0.6) (0.4.0)
Requirement already satisfied: pystan>=2.14 in /local_disk0/pythonVirtualEnvDirs/virtualEnv-4f166160-0e58-461f-9476-d73ade108c55/lib/python3.7/site-packages (from fbprophet==0.6) (2.19.1.1)
Requirement already satisfied: LunarCalendar>=0.0.9 in /local_disk0/pythonVirtualEnvDirs/virtualEnv-4f166160-0e58-461f-9476-d73ade108c55/lib/python3.7/site-packages (from fbprophet==0.6) (0.0.9)
Requirement already satisfied: convertdate>=2.1.2 in /local_disk0/pythonVirtualEnvDirs/virtualEnv-4f166160-0e58-461f-9476-d73ade108c55/lib/python3.7/site-packages (from fbprophet==0.6) (2.2.1)
Requirement already satisfied: holidays>=0.9.5 in /local_disk0/pythonVirtualEnvDirs/virtualEnv-4f166160-0e58-461f-9476-d73ade108c55/lib/python3.7/site-packages (from fbprophet==0.6) (0.10.2)
Requirement already satisfied: setuptools-git>=1.2 in /local_disk0/pythonVirtualEnvDirs/virtualEnv-4f166160-0e58-461f-9476-d73ade108c55/lib/python3.7/site-packages (from fbprophet==0.6) (1.2)
Requirement already satisfied: python-dateutil>=2.8.0 in /local_disk0/pythonVirtualEnvDirs/virtualEnv-4f166160-0e58-461f-9476-d73ade108c55/lib/python3.7/site-packages (from fbprophet==0.6) (2.8.1)
Requirement already satisfied: pytz>=2017.2 in /local_disk0/pythonVirtualEnvDirs/virtualEnv-4f166160-0e58-461f-9476-d73ade108c55/lib/python3.7/site-packages (from pandas>=0.23.2->koalas==1.0.0) (2019.3)
Requirement already satisfied: kiwisolver>=1.0.1 in /local_disk0/pythonVirtualEnvDirs/virtualEnv-4f166160-0e58-461f-9476-d73ade108c55/lib/python3.7/site-packages (from matplotlib>=3.0.0->koalas==1.0.0) (1.1.0)
Requirement already satisfied: pyparsing!=2.0.4,!=2.1.2,!=2.1.6,>=2.0.1 in /local_disk0/pythonVirtualEnvDirs/virtualEnv-4f166160-0e58-461f-9476-d73ade108c55/lib/python3.7/site-packages (from matplotlib>=3.0.0->koalas==1.0.0) (2.4.6)
Requirement already satisfied: cycler>=0.10 in /local_disk0/pythonVirtualEnvDirs/virtualEnv-4f166160-0e58-461f-9476-d73ade108c55/lib/python3.7/site-packages (from matplotlib>=3.0.0->koalas==1.0.0) (0.10.0)
Requirement already satisfied: six>=1.0.0 in /local_disk0/pythonVirtualEnvDirs/virtualEnv-4f166160-0e58-461f-9476-d73ade108c55/lib/python3.7/site-packages (from pyarrow>=0.10->koalas==1.0.0) (1.14.0)
Requirement already satisfied: ephem>=3.7.5.3 in /local_disk0/pythonVirtualEnvDirs/virtualEnv-4f166160-0e58-461f-9476-d73ade108c55/lib/python3.7/site-packages (from LunarCalendar>=0.0.9->fbprophet==0.6) (3.7.7.1)
Requirement already satisfied: pymeeus<=1,>=0.3.6 in /local_disk0/pythonVirtu

In [5]:
import databricks.koalas as ks
import fbprophet

print(ks.__version__)
print(fbprophet.__version__)

1.0.0
0.6

We will be using data detailing the occurence of flora and fauna in Australia from the start of 2010 to May 2020.

Let's first have a look at our csv file in the [Databricks file system](https://docs.databricks.com/data/databricks-file-system.html#databricks-file-system-dbfs). Note that this dataset is nearly 18GB in size.

In [7]:
%fs ls dbfs:/home/niall/koalas/sais_2020_csv/

path,name,size
dbfs:/home/niall/koalas/sais_2020_csv/gbif_australia_2010_2020.csv,gbif_australia_2010_2020.csv,17789374267


### Load full dataset using pandas

Due to the size of this dataset, we are inherently limited by the amount of memory we have on a single machine to load this dataset. The following cell outlines how we would load the Australia occurence dataset using pandas, however we have commented this out due to the fact that we run into an out of memory (OOM) error.

In [9]:
import pandas as pd

# pdf = pd.read_csv("/dbfs/home/niall/koalas/sais_2020_csv/gbif_australia_2010_2020.csv, sep="\t")
# pdf.head()
# NOTE: This cell will give an OOM error where driver memory is less than 18GB

### Load full dataset using Koalas

Let's load our dataset using Koalas. As Koalas recreates the pandas API utilising Apache Spark under the hood, we are now able to load the same dataset using Koalas' equivalent [`read_csv`](https://koalas.readthedocs.io/en/latest/reference/api/databricks.koalas.read_csv.html) function.

Note that we set the `default_index_type` to be `distributed`. We select this index type as our dataset is relatively large and we will not explicitly require an index that monotonically increases one-by-one. See [here](https://koalas.readthedocs.io/en/latest/user_guide/options.html#default-index-type) for more information on index types.

In [11]:
import databricks.koalas as ks
ks.set_option('compute.default_index_type', 'distributed')

In [12]:
# kdf = ks.read_csv("dbfs:/home/niall/koalas/sais_2020_csv/gbif_australia_2010_2020.csv", sep="\t") 
# kdf.head()

### Use Koalas to read same dataset as a Delta table

Another benefit we get from Koalas is the ability to natively load data from a [Delta Lake](https://docs.delta.io/latest/index.html) table. We can do so using the [`read_delta`](https://koalas.readthedocs.io/en/latest/reference/api/databricks.koalas.read_delta.html) function.

Note that to do this, the above csv file was separately written to disk as a Delta Lake table using Koalas' [`to_delta`](https://koalas.readthedocs.io/en/latest/reference/api/databricks.koalas.DataFrame.to_delta.html) functionality.

In [14]:
kdf = ks.read_delta("dbfs:/home/niall/koalas/sais_2020")
kdf.head()

,gbifID,datasetKey,occurrenceID,kingdom,phylum,class,order,family,genus,species,infraspecificEpithet,taxonRank,scientificName,verbatimScientificName,verbatimScientificNameAuthorship,countryCode,locality,stateProvince,occurrenceStatus,individualCount,publishingOrgKey,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,coordinatePrecision,elevation,elevationAccuracy,depth,depthAccuracy,eventDate,day,month,year,taxonKey,speciesKey,basisOfRecord,institutionCode,collectionCode,catalogNumber,recordNumber,identifiedBy,dateIdentified,license,rightsHolder,recordedBy,typeStatus,establishmentMeans,lastInterpreted,mediaType,issue
0,2431168035,8531183a-86b5-459b-a93e-37198f38f8a6,ea407c91-8bba-4ad5-a2dc-5b3742cfb31b,Fungi,Ascomycota,None,None,None,None,None,None,PHYLUM,Ascomycota,Ascomycota,None,AU,Yellabinna Regional Reserve,South Australia,present,NaN,3c5e4331-7f2f-4a8d-aa56-81ece7014fc8,-30.897414,132.224010,NaN,NaN,NaN,NaN,NaN,NaN,2016-08-12,12,8,2016,95,NaN,UNKNOWN,None,None,None,None,None,NaT,CC_BY_4_0,None,None,None,None,2020-05-06 10:02:06.589,None,BASIS_OF_RECORD_INVALID;COORDINATE_ROUNDED
1,2431985984,8531183a-86b5-459b-a93e-37198f38f8a6,ea427dc7-820d-49c4-9dd4-864d8f7d2347,Fungi,Basidiomycota,None,None,None,None,None,None,PHYLUM,Basidiomycota,Basidiomycota,None,AU,Fitzgerald River NP,Western Australia,present,NaN,3c5e4331-7f2f-4a8d-aa56-81ece7014fc8,-33.688361,119.761694,NaN,NaN,NaN,NaN,NaN,NaN,2013-12-07,7,12,2013,34,NaN,UNKNOWN,None,None,None,None,None,NaT,CC_BY_4_0,None,None,None,None,2020-05-06 10:02:06.589,None,BASIS_OF_RECORD_INVALID;COORDINATE_ROUNDED
2,2430952314,8531183a-86b5-459b-a93e-37198f38f8a6,00085092-7fd1-4751-ba21-d5170860b35d,Fungi,None,None,None,None,None,None,None,KINGDOM,Fungi,Fungi,None,AU,"Yass TB, Glenrock site",New South Wales,present,NaN,3c5e4331-7f2f-4a8d-aa56-81ece7014fc8,-34.858486,148.599397,NaN,NaN,NaN,NaN,NaN,NaN,2013-01-18,18,1,2013,5,NaN,UNKNOWN,None,None,None,None,None,NaT,CC_BY_4_0,None,None,None,None,2020-05-06 10:02:01.065,None,BASIS_OF_RECORD_INVALID
3,2430952315,8531183a-86b5-459b-a93e-37198f38f8a6,00085745-09b3-40d4-a3ca-a1868313ae46,Fungi,None,None,None,None,None,None,None,KINGDOM,Fungi,Fungi,None,AU,Mount bold,South Australia,present,NaN,3c5e4331-7f2f-4a8d-aa56-81ece7014fc8,-35.101944,138.650715,NaN,NaN,NaN,NaN,NaN,NaN,2014-12-17,17,12,2014,5,NaN,UNKNOWN,None,None,None,None,None,NaT,CC_BY_4_0,None,None,None,None,2020-05-06 10:02:01.065,None,BASIS_OF_RECORD_INVALID
4,2431563900,8531183a-86b5-459b-a93e-37198f38f8a6,ea465eca-95b8-48bb-8746-b087bf4f6c11,Fungi,Basidiomycota,None,None,None,None,None,None,PHYLUM,Basidiomycota,Basidiomycota,None,AU,Freycinet NP,Tasmania,present,NaN,3c5e4331-7f2f-4a8d-aa56-81ece7014fc8,-42.023300,148.282800,NaN,NaN,NaN,NaN,NaN,NaN,2013-07-17,17,7,2013,34,NaN,UNKNOWN,None,None,None,None,None,NaT,CC_BY_4_0,None,None,None,None,2020-05-06 10:02:06.590,None,BASIS_OF_RECORD_INVALID


We can check the size of our Koalas DataFrame using [`.shape`](https://koalas.readthedocs.io/en/latest/reference/api/databricks.koalas.DataFrame.shape.html?highlight=shape#databricks.koalas.DataFrame.shape)

In [16]:
kdf.shape

Out[6]: (35048392, 50)

Let's test an inital SQL query against our Koalas DataFrame. As Koalas is effectively a wrapper on top Spark, we are running this SQL query using the Spark SQL engine under the hood. 

[Phascolarctos](https://en.wikipedia.org/wiki/Phascolarctos) seems like an appropriate first genus to use as a query...

In [18]:
genus_test = "Phascolarctos"

ks.sql("""
SELECT COUNT(*) 
FROM {kdf}
WHERE genus = {genus_test}
""")

,count(1)
0,44202


In [19]:
# Grab the column names
kdf.columns

Out[8]: Index(['gbifID', 'datasetKey', 'occurrenceID', 'kingdom', 'phylum', 'class',
 'order', 'family', 'genus', 'species', 'infraspecificEpithet',
 'taxonRank', 'scientificName', 'verbatimScientificName',
 'verbatimScientificNameAuthorship', 'countryCode', 'locality',
 'stateProvince', 'occurrenceStatus', 'individualCount',
 'publishingOrgKey', 'decimalLatitude', 'decimalLongitude',
 'coordinateUncertaintyInMeters', 'coordinatePrecision', 'elevation',
 'elevationAccuracy', 'depth', 'depthAccuracy', 'eventDate', 'day',
 'month', 'year', 'taxonKey', 'speciesKey', 'basisOfRecord',
 'institutionCode', 'collectionCode', 'catalogNumber', 'recordNumber',
 'identifiedBy', 'dateIdentified', 'license', 'rightsHolder',
 'recordedBy', 'typeStatus', 'establishmentMeans', 'lastInterpreted',
 'mediaType', 'issue'],
 dtype='object')

# 2. Exploratory Data Analysis

Having loaded our Australia flora and fauna occurence dataset, let's first conduct some basic exploratory analysis using Koalas

The `.describe()` method in Koalas is similar to that in pandas, showing a statistical summary of each numerical feature: 
* number of non-missing values
* mean
* standard deviation
* min,max
* median
* 0.25 and 0.75 quartiles

In [23]:
kdf.describe()

,gbifID,individualCount,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,coordinatePrecision,elevation,elevationAccuracy,depth,depthAccuracy,day,month,year,taxonKey,speciesKey
count,3.504839e+07,1.663245e+07,3.503452e+07,3.503452e+07,1.046318e+07,54.000000,1.224723e+06,72333.000000,5.008753e+06,4.717326e+06,3.455704e+07,3.503676e+07,3.504839e+07,3.504839e+07,2.930489e+07
mean,1.806765e+09,5.337369e+02,-3.119349e+01,1.449740e+02,6.290393e+03,12.870370,4.857071e+02,1.315292,4.032160e+01,9.261970e-01,1.530299e+01,6.714010e+00,2.014686e+03,3.529270e+06,3.614531e+06
std,4.285418e+08,2.842948e+05,7.135650e+00,9.571929e+00,6.831663e+04,9.115705,4.107649e+02,31.936436,2.160074e+02,7.134302e+00,8.805288e+00,3.481573e+00,2.474210e+00,2.220431e+06,1.865908e+06
min,7.026291e+07,-5.000000e+00,-6.666700e+01,-1.707000e+02,2.000000e-01,0.000000,-2.028000e+03,0.000000,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,2.010000e+03,0.000000e+00,1.000109e+06
25%,1.453812e+09,1.000000e+00,-3.583088e+01,1.443714e+02,2.000000e+01,1.000000,1.120000e+02,0.000000,2.500000e+00,0.000000e+00,8.000000e+00,4.000000e+00,2.013000e+03,2.480331e+06,2.481161e+06
50%,1.819640e+09,2.000000e+00,-3.368467e+01,1.473971e+02,1.000000e+02,20.000000,4.210000e+02,0.000000,5.500000e+00,0.000000e+00,1.500000e+01,7.000000e+00,2.015000e+03,2.489450e+06,2.489450e+06
75%,2.121719e+09,5.000000e+00,-2.753139e+01,1.511983e+02,1.000000e+03,20.000000,7.660000e+02,0.000000,2.500000e+01,0.000000e+00,2.300000e+01,1.000000e+01,2.017000e+03,5.229134e+06,5.228220e+06
max,2.612409e+09,1.000000e+09,5.082135e+01,1.743350e+02,4.470396e+06,20.000000,1.419700e+04,2005.000000,4.985000e+03,3.027500e+03,3.100000e+01,1.200000e+01,2.020000e+03,1.066254e+07,1.066214e+07


### Null Values

In [25]:
kdf.isna().sum()

Out[10]: gbifID 0
datasetKey 0
occurrenceID 8866
kingdom 0
phylum 755292
class 1221570
order 1760770
family 2267625
genus 3374303
species 5743500
infraspecificEpithet 34636628
taxonRank 0
scientificName 0
verbatimScientificName 37586
verbatimScientificNameAuthorship 34955574
countryCode 0
locality 2465014
stateProvince 5448608
occurrenceStatus 20839415
individualCount 18415946
publishingOrgKey 3
decimalLatitude 13875
decimalLongitude 13875
coordinateUncertaintyInMeters 24585217
coordinatePrecision 35048338
elevation 33823669
elevationAccuracy 34976059
depth 30039639
depthAccuracy 30331066
eventDate 3
day 491351
month 11629
year 3
taxonKey 3
speciesKey 5743503
basisOfRecord 3
institutionCode 11842331
collectionCode 13996731
catalogNumber 9770905
recordNumber 33062515
identifiedBy 34327005
dateIdentified 34414329
license 3
rightsHolder 34091201
recordedBy 10985200
typeStatus 35047620
establishmentMeans 35045299
lastInterpreted 3
mediaType 34151401
issue 13358379
Name: 0, dtype: int64

A lot of our following analysis will be based off the counts of given plant and animal species. As such, let's keep only those observations where `individualCount` is not null.

In [27]:
kdf = kdf[kdf["individualCount"].notnull() & kdf["month"].notnull() & kdf["year"].notnull()]

### Value Counts

Looking at the value counts for the `kingdom` column

In [30]:
kdf["kingdom"].value_counts(normalize=True)

Out[12]: Animalia 9.532980e-01
Plantae 4.429007e-02
incertae sedis 1.186183e-03
Chromista 8.286017e-04
Fungi 3.644380e-04
Protozoa 2.075113e-05
Bacteria 1.178905e-05
Viruses 1.202964e-07
Name: kingdom, dtype: float64

### Data visualization using Koalas

In [32]:
kdf["kingdom"].value_counts(normalize=True).plot.bar(rot=25, figsize=(12,8), title="Bar plot of kingdom column using Koalas DataFrame")

### Columnar analysis

Let's look more closely at the `class` column in particular.

In [34]:
# Get the number of unique classes
len(kdf["class"].unique())

Out[14]: 131

Get the total count, the number of distinct recordings and the average count per recording for each class.

In [36]:
# Group by the class column, sum up the individual counts for each group, and get the count of instances in each group.
# Note that using as_index=False resets the index
class_ks_df = kdf.groupby("class", as_index=False).agg({"individualCount": "sum", "gbifID": "count"})

# Renaming our columns to reflect the grouping aggregates
class_ks_df.columns = ["class", "total_count", "number_distinct_recordings"]

# Create a column which calculates the average number of animal/plant counts per each distinct recording for each class
class_ks_df["avg_count_per_recording"] = class_ks_df["total_count"]/class_ks_df["number_distinct_recordings"]
class_ks_df.sort_values(by="number_distinct_recordings", ascending=False).head(10)

,class,total_count,number_distinct_recordings,avg_count_per_recording
77309411328,Aves,134105503,14784786,9.070507
317827579904,Magnoliopsida,42578891,540519,78.774087
1468878815233,Actinopterygii,10775303,413427,26.063375
257698037760,Mammalia,92025885,289830,317.516768
51539607552,Liliopsida,26554000,173298,153.227389
687194767360,Reptilia,267173,118182,2.260691
214748364800,Insecta,2460706,98031,25.101305
188978561024,Amphibia,556740,68052,8.181097
360777252864,None,6485438612,25289,256452.948397
317827579905,Arachnida,144777,25067,5.775601


Let's look at the distribution of the average counts per recording using [`.hist()`](https://koalas.readthedocs.io/en/latest/reference/api/databricks.koalas.Series.hist.html?highlight=hist#databricks-koalas-series-hist)

In [38]:
class_ks_df["avg_count_per_recording"].hist(bins=100, figsize=(12,8), title="Histogram of average individualCount per observation (Aggregated to class level)")

We see that the distribution of the average counts per observation is very skewed, with the majority of classes only having a small number of instances per 5x5km recording.

On the converse, we see there are a number of classes with a very high average count per recording. Let's investigate...

In [40]:
class_ks_df.sort_values(by="avg_count_per_recording", ascending=False).head().style.background_gradient(cmap="Greens")

,class,total_count,number_distinct_recordings,avg_count_per_recording
618475290625,Cyanophyceae,134028801,193,694449.746114
850403524608,Dinophyceae,1022228109,2439,419117.715867
360777252864,None,6485438612,25289,256452.948397
120259084288,Prymnesiophyceae,21843254,89,245429.820225
970662608897,Raphidophyceae,1166650,5,233330.000000


For the [Cyanophyceae](https://www.gbif.org/species/144093497) class, on average there are nearly 7 million instances per recording. Understandable given the microscopic nature of algae!

# 3. Analysis of Koala Numbers in Australia

### Koala *(genus: Phascolarctos)* 🐨

In [Australia](https://www.savethekoala.com/about-koalas/koala-endangered-or-not), koala populations have unfortunately been declining over recent times due to the effects of habitat destruction, bushfires, domestic dog attacks and road accidents. Let's analyze the occurences of koalas in our Australian dataset and see if the data confirms such reports.

![](https://media.mnn.com/assets/images/2019/05/koala.jpg.653x0_q80_crop-smart.jpg)

In [44]:
# Filter down to the genus Phascolarctos
koalas_df = kdf[kdf.genus == "Phascolarctos"]
koalas_df.head()

,gbifID,datasetKey,occurrenceID,kingdom,phylum,class,order,family,genus,species,infraspecificEpithet,taxonRank,scientificName,verbatimScientificName,verbatimScientificNameAuthorship,countryCode,locality,stateProvince,occurrenceStatus,individualCount,publishingOrgKey,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,coordinatePrecision,elevation,elevationAccuracy,depth,depthAccuracy,eventDate,day,month,year,taxonKey,speciesKey,basisOfRecord,institutionCode,collectionCode,catalogNumber,recordNumber,identifiedBy,dateIdentified,license,rightsHolder,recordedBy,typeStatus,establishmentMeans,lastInterpreted,mediaType,issue
0,1935533732,0645ccdb-e001-4ab0-9729-51f1755e007e,258731e5-c725-4012-ae5d-3e719be12d99,Animalia,Chordata,Mammalia,Diprotodontia,Phascolarctidae,Phascolarctos,Phascolarctos cinereus,None,SPECIES,"Phascolarctos cinereus (Goldfuss, 1817)",Phascolarctos cinereus,None,AU,locality withheld,New South Wales,present,1,5eee35c2-c5b7-4f62-a527-09eec0c54f22,-32.721543,151.720771,10.0,NaN,NaN,NaN,NaN,NaN,2018-02-17,17,2,2018,2440012,2440012,HUMAN_OBSERVATION,NSW Office of Environment and Heritage,BioNet Atlas of NSW Wildlife,SJJSI0770276,5894,None,NaT,CC_BY_4_0,None,OJJS1208150J,None,None,2020-05-08 01:21:26.830,None,None
1,1452759523,0645ccdb-e001-4ab0-9729-51f1755e007e,60a4e3aa-6e01-4ee0-b649-5726c05149b6,Animalia,Chordata,Mammalia,Diprotodontia,Phascolarctidae,Phascolarctos,Phascolarctos cinereus,None,SPECIES,"Phascolarctos cinereus (Goldfuss, 1817)",Phascolarctos cinereus,None,AU,locality withheld,New South Wales,present,3,5eee35c2-c5b7-4f62-a527-09eec0c54f22,-34.410028,150.534459,5.0,NaN,NaN,NaN,NaN,NaN,2015-11-16,16,11,2015,2440012,2440012,HUMAN_OBSERVATION,NSW Office of Environment and Heritage,BioNet Atlas of NSW Wildlife,SMZS1602120E,None,None,NaT,CC_BY_4_0,None,OMZS16021200,None,None,2020-05-08 01:21:44.095,None,COORDINATE_ROUNDED
2,1452164639,0645ccdb-e001-4ab0-9729-51f1755e007e,bf268ddc-3425-4f10-85dd-aa7890212592,Animalia,Chordata,Mammalia,Diprotodontia,Phascolarctidae,Phascolarctos,Phascolarctos cinereus,None,SPECIES,"Phascolarctos cinereus (Goldfuss, 1817)",Phascolarctos cinereus,None,AU,locality withheld,New South Wales,present,1,5eee35c2-c5b7-4f62-a527-09eec0c54f22,-32.710725,152.065533,10.0,NaN,NaN,NaN,NaN,NaN,2015-02-10,10,2,2015,2440012,2440012,HUMAN_OBSERVATION,NSW Office of Environment and Heritage,BioNet Atlas of NSW Wildlife,SIXRI1136353,97,None,NaT,CC_BY_4_0,None,OJJS1208150Z,None,None,2020-05-08 01:20:55.287,None,None
3,1571052240,0645ccdb-e001-4ab0-9729-51f1755e007e,2b361547-6a7c-43b8-a5cb-ab17536d7329,Animalia,Chordata,Mammalia,Diprotodontia,Phascolarctidae,Phascolarctos,Phascolarctos cinereus,None,SPECIES,"Phascolarctos cinereus (Goldfuss, 1817)",Phascolarctos cinereus,None,AU,locality withheld,New South Wales,present,1,5eee35c2-c5b7-4f62-a527-09eec0c54f22,-28.386206,153.537519,50.0,NaN,NaN,NaN,NaN,NaN,2016-08-18,18,8,2016,2440012,2440012,HUMAN_OBSERVATION,NSW Office of Environment and Heritage,BioNet Atlas of NSW Wildlife,SJJSI0292090,None,None,NaT,CC_BY_4_0,None,OJJS07060700,None,None,2020-05-08 01:21:44.911,None,COORDINATE_ROUNDED
4,2421476691,0645ccdb-e001-4ab0-9729-51f1755e007e,4195682f-2855-418b-bfe6-891df61823e9,Animalia,Chordata,Mammalia,Diprotodontia,Phascolarctidae,Phascolarctos,Phascolarctos cinereus,None,SPECIES,"Phascolarctos cinereus (Goldfuss, 1817)",Phascolarctos cinereus,None,AU,locality withheld,New South Wales,present,1,5eee35c2-c5b7-4f62-a527-09eec0c54f22,-32.735673,152.082932,10.0,NaN,NaN,NaN,NaN,NaN,2016-07-22,22,7,2016,2440012,2440012,HUMAN_OBSERVATION,NSW Office of Environment and Heritage,BioNet Atlas of NSW Wildlife,SPJGI5927420,OEH:WR520888; org:5455,None,NaT,CC_BY_4_0,None,OJJS17051722,None,None,2020-05-08 01:21:44.358,None,COORDINATE_ROUNDED


In [45]:
koalas_df.shape[0]

Out[19]: 21919

We have 21,919 instances of koalas recordings in the dataset, after removing null values for `individualCount`, `month` and `day`.

Let's check the date range for which we have recordings of koala numbers

In [48]:
koalas_df["eventDate"].min(), koalas_df["eventDate"].max()

Out[20]: (datetime.datetime(2010, 1, 1, 0, 0), datetime.datetime(2020, 5, 1, 0, 0))

How many observations were recorded by each province from 2010 to May 2020?

In [50]:
koalas_df["stateProvince"].value_counts()

Out[21]: New South Wales 15315
Queensland 3156
South Australia 2763
Victoria 670
Northern Territory 6
Australian Capital Territory 6
Name: stateProvince, dtype: int64

What is the distribution of the individual count of Koalas for each observation?

In [52]:
koalas_df["individualCount"].hist(bins=100, figsize=(12,8), title="Histogram of individualCount per recording for Koalas")

We note that the majority of observations record a small number of Koalas, however there is a tiny minority of instances where there are a huge number of Koalas recorded - let's treat these as outliers and remove any observations where over 100 koalas have been recorded for a single recording.

In [54]:
koalas_df = koalas_df[koalas_df["individualCount"] < 100]

koalas_df["individualCount"].hist(bins=100, figsize=(12,8), title="Histogram of individualCount per recording for Koalas")

To get an idea of the overall population of koalas in Australia since 2010, let's compute the number of occurences for koalas recorded, along with the total number of koalas counted and get the average number of koalas per recorded sighting.

In [56]:
# Get the sum of all koalas recorded in Austrailia for each month
koalas_sum_ks_df = koalas_df.groupby(["year", "month"], as_index=False)["individualCount"].sum()

# Get the count of all koalas occurences in the Austrailia for each month
koalas_count_ks_df = koalas_df.groupby(["year", "month"], as_index=False)["individualCount"].count()

# Join the above Koalas DataFrames and divide the total_count column by num_recordings column
koalas_avg_cnt_ks_df = koalas_sum_ks_df.merge(koalas_count_ks_df, on=["year", "month"])
koalas_avg_cnt_ks_df.columns = ["year", "month", "total_count", "num_recordings"]

# Get number of koalas per recording nationally
koalas_avg_cnt_ks_df["avg_cnt"] = koalas_avg_cnt_ks_df["total_count"]/koalas_avg_cnt_ks_df["num_recordings"]

koalas_avg_cnt_ks_df.sort_values(by=["year", "month"], ascending=False).head(12)

,year,month,total_count,num_recordings,avg_cnt
1563368095744,2020,5,3,2,1.500000
833223655425,2020,4,37,34,1.088235
240518168576,2020,3,25,21,1.190476
1340029796354,2020,2,27,25,1.080000
1039382085633,2020,1,86,69,1.246377
1262720385025,2019,12,109,84,1.297619
1554778161152,2019,11,86,75,1.146667
68719476736,2019,10,87,74,1.175676
1159641169920,2019,9,202,166,1.216867
472446402561,2019,8,148,132,1.121212


### Plotting Australian koalas numbers over time   

To investigate koalas numbers over time let's plot the count per month of koalas from 2010 to May 2020. To do so, we're first going to combine our year and month columns to create a monthly datetime column.

In [58]:
# Concatenating the year and month columns to create a year_month column
koalas_avg_cnt_ks_df = ks.sql("""
SELECT year, month, num_recordings, total_count, avg_cnt, CONCAT(year, "-", month) AS year_month
FROM {koalas_avg_cnt_ks_df}
""")

koalas_avg_cnt_ks_df.head()

,year,month,num_recordings,total_count,avg_cnt,year_month
17179869184,2012,10,200,251,1.255000,2012-10
120259084288,2017,8,219,229,1.045662,2017-8
128849018880,2014,4,83,172,2.072289,2014-4
188978561024,2012,8,143,318,2.223776,2012-8
188978561025,2019,3,175,208,1.188571,2019-3


In [59]:
# Convert the year_month column to datetime (returns in format %Y-%m-%d)
koalas_avg_cnt_ks_df["year_month"] = ks.to_datetime(koalas_avg_cnt_ks_df["year_month"])

# Need to sort by year_month before plotting
koalas_avg_cnt_ks_df.sort_values(by=['year_month'], inplace=True)

/databricks/spark/python/pyspark/sql/pandas/functions.py:386: UserWarning:

In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

We will create NumPy arrays of the columns we would like to pass into our `matplotlib` plot. Note that this method should only be used if the resulting NumPy ndarray is expected to be small, as all the data is loaded into the driver’s memory.

In [61]:
# Creating numpy arrays from year_month, total_count and num_recordings
x_axis_vals = koalas_avg_cnt_ks_df["year_month"].to_numpy()
y1_vals = koalas_avg_cnt_ks_df["total_count"].to_numpy()
y2_vals = koalas_avg_cnt_ks_df["num_recordings"].to_numpy()

Let's first plot monthly total counts and the number of monthly recordings for koalas from 2010 until May 2020.

In [63]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style("white")
plt.rcParams.update({'font.size': 9})

fig, ax1 = plt.subplots(figsize=(12,8))

color = 'tab:red'
ax1.set_xlabel('year_month')
ax1.set_ylabel('total_count', color=color)
ax1.plot(x_axis_vals, y1_vals, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx() 
color = 'tab:blue'
ax2.set_ylabel('num_recordings', color=color) 
ax2.plot(x_axis_vals, y2_vals, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  
ax1.set_title("Monthly total count and number of recordings for Koalas in Australia from 2011 - 2018", size=9)

plt.show()

To get a true reflection of koala numbers over time, let's plot the average number of koalas per recording for each month between 2010 and May of 2020.

In [65]:
sns.set_style("whitegrid")

# Set year_month to be the index and keep only avg_cnt
koalas_avg_count_ks_series = koalas_avg_cnt_ks_df.set_index("year_month")["avg_cnt"]

plt.title("Monthly average koala count in Australia from 2010 - 2020", size=9)
plt.xlabel("date")
plt.ylabel("avg_num_koalas_per_occurence")

koalas_avg_count_ks_series.plot(figsize=(12,8))

# 4. Parallelizing Model Training with Koalas

### Forecasting with Prophet

Let's use our Australian flora and fauna dataset to forecast future total numbers for `genus` on a monthly basis. To do so, we will use Facebook's open source time-series forecasting library, [Prophet](https://facebook.github.io/prophet/). 

Our aim here is not to fit a highly accurate forecasting model, but more so demonstrating how one has the flexibility to train numerous models in parallel utilizing Koalas.

In [68]:
# First filter to the classes we would like to forecast for
filter_1 = (kdf["genus"] == "Phascolarctos") # Koala
filter_2 = (kdf["genus"] == "Macropus") # Kangaroo
filter_3 = (kdf["genus"] == "Cacatua") # Cockatoo
filter_4 = (kdf["genus"] == "Dromaius") # Emu

subset_kdf = kdf[filter_1 | filter_2 | filter_3 | filter_4]

# Get the total number of counts and total number of observations for each class in every month
grouped_kdf = subset_kdf.groupby(["genus", "year", "month"], as_index=False) \
                          .agg({"individualCount": "sum", "occurrenceID": "count"}) \
                          .sort_values(by=["year", "month"])

grouped_kdf.columns = ["genus", "year", "month", "total_count", "num_recordings"]

# Concatenating the year and month columns to create a year_month column
grouped_kdf = ks.sql("""
SELECT genus, year, month, num_recordings, total_count, CONCAT(year, "-", month) AS year_month
FROM {grouped_kdf}
ORDER BY year, month
""")

# Convert the year_month column to datetime (returns in format %Y-%m-%d)
grouped_kdf["year_month"] = ks.to_datetime(grouped_kdf["year_month"], format="%Y-%m")
grouped_kdf.head()

,genus,year,month,num_recordings,total_count,year_month
0,Phascolarctos,2010,1,128,143,2010-01-01
1,Macropus,2010,1,39,1786,2010-01-01
2,Cacatua,2010,1,1129,90213,2010-01-01
3,Dromaius,2010,1,6,78,2010-01-01
17179869184,Phascolarctos,2010,3,119,141,2010-03-01


To parallelize the training of each individual Prophet model per genus, we will utilize Koalas [groupby-apply](https://koalas.readthedocs.io/en/latest/reference/api/databricks.koalas.groupby.GroupBy.apply.html#databricks-koalas-groupby-groupby-apply) method. 

Here we demonstrate that we can apply an arbitrary python function to a group of our data, for example in this instance for each genus,  we apply the `fcst_func` function we have defined which fits a Prophet model and logs parameters and plots to MLflow. We thus pass in a Koalas DataFrame, to have a Koalas DataFrame returned, however note that the returned koalas.DataFrame can have different number rows and columns as the input.

Under the hood, Koalas makes use of Spark's [pandas UDF](https://spark.apache.org/docs/latest/sql-pyspark-pandas-with-arrow.html#grouped-map) functionality when implementing a groupby-apply method. As such, the data within the function being applied is actually a pandas DataFrame. Resultantly, any pandas APIs within this function are allowed.

When calling groupby-apply, Koalas executes the function once for a small sample to infer the type which can be potentially expensive; for example, where a dataset is created after aggregations or sorting. To avoid this, we specify a return type hint in function being applied.

When forecasting for each individual genus we will produce a number of different plots. To record these, we will use [MLflow](https://mlflow.org/docs/latest/index.html) to log them as artifacts.

In [70]:
# MLflow set up
import mlflow
from  mlflow.tracking import MlflowClient

# MLflow experiment (specify path for experiment to be set to)
# NOTE: you will need to change the following experiment path 
# dependent on where you would like to save the model to
mlflow_exp = f"/Users/niall.turbitt@databricks.com/Koalas/Koalas SAIS 2020/koalas_sais20_exp"
mlflow.set_experiment(mlflow_exp)

# Get the experiment ID to track to
exp_id = MlflowClient().get_experiment_by_name(mlflow_exp).experiment_id

In [71]:
from fbprophet import Prophet
import numpy as np
import matplotlib.pyplot as plt

def fcst_func(pdf) -> pd.DataFrame[np.datetime64, float, float, int, str]:
  """
  Function to take grouped genus data, and provide 3-year monthly 
  forecast of total count. Log model plot as artifacts in MLflow 
  
  pdf: grouped genus data
  """ 
  genus_name = pdf["genus"].iloc[0]
  pdf["ds"] = pdf["year_month"]
  pdf["y"] = pdf["total_count"]
  df = pdf[["ds", "y"]].copy()
    
  # log the run with MLflow
  with mlflow.start_run(experiment_id=exp_id,
                        run_name=f"{genus_name} - Prophet Forecast") as run:
    
    prophet_daily_seasonality = False
    prophet_weekly_seasonality = False
    prophet_yearly_seasonality = True
    
    mlflow.log_params({"genus": genus_name,
                       "prophet_param_daily_seasonality": prophet_daily_seasonality,
                       "prophet_param_weekly_seasonality": prophet_weekly_seasonality,
                       "prophet_param_yearly_seasonality": prophet_yearly_seasonality})

    m = Prophet(daily_seasonality = prophet_daily_seasonality, 
                weekly_seasonality = prophet_weekly_seasonality, 
                yearly_seasonality = prophet_yearly_seasonality, 
                seasonality_mode = "additive")
    m.fit(df)
    future = m.make_future_dataframe(periods=36, freq="M")
    fcst = m.predict(future)

    fig_1 = m.plot(fcst)
    plt.xlabel(f"Date")
    plt.ylabel(f"{genus_name} Numbers (Monthly)")
    plt.title(f"{genus_name} - Prophet Forecast")
    fcst_path = f"/tmp/{genus_name}_forecast.png"
    fig_1.savefig(fcst_path)
    mlflow.log_artifact(fcst_path)
  
    fig_2 = m.plot_components(fcst)
    components_path = f"/tmp/{genus_name}_components.png"
    fig_2.savefig(components_path)
    mlflow.log_artifact(components_path)
    plt.clf()
    
  ret_pdf = fcst[["ds", "yhat", "yhat_lower", "yhat_upper"]]
  ret_pdf["genus"] = [genus_name]*len(ret_pdf)

  return ret_pdf

In [72]:
fcst_df = grouped_kdf.groupby("genus", as_index=False).apply(fcst_func)

Note that the above will only trigger a Spark job under the hood if we call an action. Thus we compute the length of resulting DataFrame.

In [74]:
len(fcst_df)

Out[45]: 637

We can search the runs of our MLflow experiment programmatically using [`search_runs`](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.search_runs)

In [76]:
test_genus = "Cacatua"

# The following returns a pandas DataFrame of runs
mlflow_res_pdf = mlflow.search_runs(filter_string=f"params.genus='{test_genus}'").sort_values("start_time", ascending=False)
mlflow_res_pdf

,run_id,experiment_id,status,artifact_uri,start_time,end_time,params.prophet_param_yearly_seasonality,params.prophet_param_weekly_seasonality,params.prophet_param_daily_seasonality,params.genus,tags.mlflow.user,tags.mlflow.databricks.notebookRevisionID,tags.mlflow.source.name,tags.mlflow.databricks.notebookPath,tags.mlflow.runName,tags.mlflow.source.type,tags.mlflow.databricks.notebookID,tags.mlflow.databricks.webappURL
0,ea6cda2aed5849f4b8618a775bf81120,7046879,FINISHED,dbfs:/databricks/mlflow/7046879/ea6cda2aed5849...,2020-06-19 10:51:06.346000+00:00,2020-06-19 10:51:12.150000+00:00,True,False,False,Cacatua,niall.turbitt@databricks.com,1592563872230,/Users/niall.turbitt@databricks.com/Koalas/Koa...,/Users/niall.turbitt@databricks.com/Koalas/Koa...,Cacatua - Prophet Forecast,NOTEBOOK,6851176,https://demo.cloud.databricks.com
1,bff35a3b0bf747418c38f88723a3cdc1,7046879,FINISHED,dbfs:/databricks/mlflow/7046879/bff35a3b0bf747...,2020-06-19 10:50:47.881000+00:00,2020-06-19 10:50:53.050000+00:00,True,False,False,Cacatua,niall.turbitt@databricks.com,1592563853134,/Users/niall.turbitt@databricks.com/Koalas/Koa...,/Users/niall.turbitt@databricks.com/Koalas/Koa...,Cacatua - Prophet Forecast,NOTEBOOK,6851176,https://demo.cloud.databricks.com
2,9aefd56ee29d4db7bcd2bf921de3b33a,7046879,FINISHED,dbfs:/databricks/mlflow/7046879/9aefd56ee29d4d...,2020-06-07 19:11:43.125000+00:00,2020-06-07 19:11:48.844000+00:00,True,False,False,Cacatua,niall.turbitt@databricks.com,1591557109239,/Users/niall.turbitt@databricks.com/Koalas/Koa...,/Users/niall.turbitt@databricks.com/Koalas/Koa...,Cacatua - Prophet Forecast,NOTEBOOK,6851176,https://demo.cloud.databricks.com


From our runs DataFrame, we can grab the artifact URI and load the forecast plot for that given run.

In [78]:
artifact_uri = mlflow_res_pdf.iloc[0]["artifact_uri"]

local_img_path = "/" + artifact_uri.replace(":","") + f"/{test_genus}_forecast.png"

plt.figure(figsize = (12,8))
img = plt.imread(local_img_path)
plt.imshow(img)
plt.show()